# ArcFace: Additive Angular Margin Loss for Deep Face Recognition.  
[click here](https://arxiv.org/abs/1801.07698)  

In [19]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import math

In [20]:
class My_Custom_Generator(tf.keras.utils.Sequence) :
  
  def __init__(self, X, labels, batch_size) :
    self.X = X
    self.labels = labels
    self.batch_size = batch_size
    self.shuffle = True
    self.on_epoch_end()

  def __iter__(self):
    """Create a generator that iterate over the Sequence."""
    for item in (self[i] for i in range(len(self))):
      yield item  
    
  def __len__(self) :
    #self.on_epoch_end()
    result = (np.ceil(len(self.X) / float(self.batch_size))).astype(np.int)
    return result

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.labels))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)
  
  def __getitem__(self, idx) :
    indexes = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
    batch_x = [self.X[k] for k in indexes]
    batch_y = [self.labels[k] for k in indexes] 
    X = np.array(batch_x, dtype=np.int32)
    Y = np.array(batch_y)
    X = np.concatenate((X, Y), axis=1)
    print(X.shape)
     
    return X, Y

In [21]:
class ArcMarginPenaltyLogists(layers.Layer):
    """ArcMarginPenaltyLogists"""
    def __init__(self, num_classes, margin=0.5, logist_scale=64, **kwargs):
        super(ArcMarginPenaltyLogists, self).__init__(**kwargs)
        self.num_classes = num_classes
        self.margin = margin
        self.logist_scale = logist_scale

    def build(self, input_shape):
        self.w = self.add_variable(
            "weights", shape=[int(input_shape[-1]), self.num_classes])
        self.cos_m = tf.identity(math.cos(self.margin), name='cos_m')
        self.sin_m = tf.identity(math.sin(self.margin), name='sin_m')
        self.th = tf.identity(math.cos(math.pi - self.margin), name='th')
        self.mm = tf.multiply(self.sin_m, self.margin, name='mm')

    def call(self, embds, labels):
        normed_embds = tf.nn.l2_normalize(embds, axis=1, name='normed_embd')
        normed_w = tf.nn.l2_normalize(self.w, axis=0, name='normed_weights')

        cos_t = tf.matmul(normed_embds, normed_w, name='cos_t')
        sin_t = tf.sqrt(1. - cos_t ** 2, name='sin_t')

        cos_mt = tf.subtract(
            cos_t * self.cos_m, sin_t * self.sin_m, name='cos_mt')

        cos_mt = tf.where(cos_t > self.th, cos_mt, cos_t - self.mm)

        mask = tf.one_hot(tf.cast(labels, tf.int32), depth=self.num_classes,
                          name='one_hot_mask')

        logists = tf.where(mask == 1., cos_mt, cos_t)
        logists = tf.multiply(logists, self.logist_scale, 'arcface_logist')

        return logists

In [22]:
num_patterns = 2000
num_features = 2
mu, sigma = 0, 0.5
num_classes = 2
num_hidden_neurons = 4
Xred = np.random.normal(mu, sigma, (num_patterns, num_features)) + np.array([1,1])
Yred = np.zeros(num_patterns, dtype=int)
Xblue = np.random.normal(mu, sigma, (num_patterns, num_features)) + np.array([-1,-1])
Yblue = np.ones(num_patterns, dtype=int)
X = np.concatenate((Xred, Xblue), axis=0)
Y = np.concatenate((Yred, Yblue), axis=0)
Y = np.reshape(Y, (-1,1))

In [23]:
inputs = tf.keras.Input(shape=(num_features+num_classes))
output_1 = layers.Dense(64, activation='relu')(inputs[:,0:num_features])
output_2 = layers.Dense(64, activation='relu')(output_1)
predictions = ArcMarginPenaltyLogists(num_classes)(output_2, inputs[:,num_features: num_features+num_classes])  
model = tf.keras.Model(inputs, predictions)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=34)
batch_size = 100
num_epochs = 50
my_training_batch_generator = My_Custom_Generator(X_train, y_train, batch_size)
my_validation_batch_generator = My_Custom_Generator(X_test, y_test, batch_size)

model.compile(
  'adam',
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=['accuracy']
)

model.fit(
  my_training_batch_generator,
  epochs=num_epochs,
  validation_data = my_validation_batch_generator,
  validation_steps=len(my_validation_batch_generator)  
)

ccuracy: 0.0000e+00
Epoch 30/50
(100, 3)
(100, 3)
 1/36 [..............................] - ETA: 0s - loss: 9.4657 - accuracy: 0.0000e+00(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
 9/36 [======>.......................] - ETA: 0s - loss: 9.7346 - accuracy: 0.0000e+00(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
17/36 [=============>................] - ETA: 0s - loss: 9.7656 - accuracy: 0.0000e+00(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
24/36 [===================>..........] - ETA: 0s - loss: 9.6173 - accuracy: 0.0000e+00(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
31/36 [========================>.....] - ETA: 0s - loss: 9.6116 - accuracy: 0.0000e+00(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
(100, 3)
36/36 [==============================] - 0s 10ms/step - loss: 9.6270 - accuracy: 0.0000e+00 - val_loss: 9.4201 - val_accuracy: 0.0000e+00
Epoch 31/50
(100, 3)
(100, 3)
 1/36 [...